# age|gender预测-统计特征版

In [2]:
import numpy as np
import pandas as pd

In [158]:
import gc

## 载入数据

### 用户日志

In [3]:
df_user_click_log_train = pd.read_csv("./data/train_preliminary/click_log.csv")
df_user_click_log_test = pd.read_csv("./data/test/click_log.csv")

In [4]:
df_user_click_log_train.shape,df_user_click_log_test.shape

((30082771, 4), (33585512, 4))

In [5]:
df_user_click_log_train.head()

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1


In [6]:
df_user_click_log_test.isnull().sum()

time           0
user_id        0
creative_id    0
click_times    0
dtype: int64

In [7]:
df_user_click_log_train['user_id'].drop_duplicates().shape,df_user_click_log_test['user_id'].drop_duplicates().shape

((900000,), (1000000,))

In [73]:
df_user_click_log_test['user_id'].drop_duplicates().values

array([3131989, 3131990, 3131994, ..., 3131984, 3131987, 3131988])

In [20]:
df_user_click_log['click_times'].describe(percentiles=[0.05,0.25,0.5,0.75,0.95])

count    3.008277e+07
mean     1.063127e+00
std      2.982831e-01
min      1.000000e+00
5%       1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
95%      2.000000e+00
max      1.520000e+02
Name: click_times, dtype: float64

### 广告信息

In [8]:
df_ad_info_train = pd.read_csv("./data/train_preliminary/ad.csv")
df_ad_info_test = pd.read_csv("./data/test/ad.csv")

In [9]:
df_ad_info_train.shape,df_ad_info_test.shape

((2481135, 6), (2618159, 6))

In [ ]:
df_ad_info.head()

In [13]:
df_ad_info.isnull().sum()

creative_id         0
ad_id               0
product_id          0
product_category    0
advertiser_id       0
industry            0
dtype: int64

In [12]:
df_ad_info[df_ad_info['product_id']!='\\N'].head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
9,20,20,34647,5,312,267
14,27,27,59,1,62948,254
21,46,46,1263,1,1274,247
33,63,63,66,1,62928,242
34,65,64,4573,3,22,141


#### 分类信息查看

In [9]:
df_ad_info_train['product_category'].value_counts().sort_index()

1        259
2     923994
3     283817
4      22917
5     227970
6      12077
7       7611
8     233832
9       2970
10        18
11      1379
12     19592
13     38072
14        95
15       332
16      7216
17     35638
18    663346
Name: product_category, dtype: int64

In [10]:
df_ad_info_test['product_category'].value_counts().sort_index()

1        277
2     974647
3     298067
4      23927
5     234806
6      13189
7       7778
8     256240
9       3159
10        13
11      1499
12     20132
13     39621
14       105
15       355
16      7457
17     36712
18    700175
Name: product_category, dtype: int64

In [13]:
df_ad_info_train['industry'].value_counts().sort_index(ascending=False).head(20)

\N    101048
99      1571
98      3461
97         3
96         1
94         1
93         2
92         5
90        53
9        342
89      7756
88     13587
87      5915
86     11565
85       930
84     11723
83         2
82      6680
81      3429
8       6146
Name: industry, dtype: int64

In [12]:
df_ad_info_test['industry'].value_counts().sort_index(ascending=False).head(20)

\N    104764
99      1650
98      3911
97         3
96         2
95         1
93         3
92         5
91         1
90        62
9        375
89      8227
88     13918
87      6199
86     12688
85      1012
84     12843
83         2
82      6898
81      3699
Name: industry, dtype: int64

查看训练集和测试集不相同的id

In [14]:
set(df_ad_info_train['industry'].value_counts().index)-set(df_ad_info_test['industry'].value_counts().index)

{'152', '266', '282', '325', '42', '94'}

In [15]:
set(df_ad_info_test['industry'].value_counts().index)-set(df_ad_info_train['industry'].value_counts().index)

{'153', '198', '199', '7', '91', '95'}

In [42]:
df_ad_info['industry'].min(),df_ad_info['industry'].max()

('1', '\\N')

In [18]:
df_ad_info['advertiser_id'].value_counts()

14681    26906
14056    24500
14682    18915
9058     12127
32876     8633
         ...  
50063        1
13201        1
5013         1
31640        1
2049         1
Name: advertiser_id, Length: 52090, dtype: int64

#### 处理缺失值

In [7]:
df_ad_info_train.replace('\\N','unknown',inplace=True)
df_ad_info_test.replace('\\N','unknown',inplace=True)

In [8]:
df_ad_info_test['industry'].value_counts()

247        275504
319        189526
322        168259
6          168083
unknown    104764
            ...  
78              1
153             1
269             1
199             1
268             1
Name: industry, Length: 326, dtype: int64

### 标签信息

In [11]:
df_user_label = pd.read_csv("./data/train_preliminary/user.csv")

In [12]:
df_user_label.shape

(900000, 3)

In [63]:
df_user_label.head()

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1


In [21]:
df_user_label['age'].value_counts()

3     202909
4     150578
2     149271
5     130667
6     101720
7      66711
1      35195
8      31967
9      19474
10     11508
Name: age, dtype: int64

In [22]:
df_user_label['gender'].value_counts()

1    602610
2    297390
Name: gender, dtype: int64

## 合并训练集和测试集点击日志

In [13]:
df_user_features_train = pd.merge(df_user_click_log_train,df_ad_info_train,on='creative_id')

In [14]:
df_user_click_log_train.shape,df_user_features_train.shape

((30082771, 4), (30082771, 9))

In [15]:
df_user_features_train['creative_id'].drop_duplicates().shape,df_user_features_train['ad_id'].drop_duplicates().shape,df_user_features_train['advertiser_id'].drop_duplicates().shape

((2481135,), (2264190,), (52090,))

In [25]:
df_user_features_train.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,15,320815,567330,1,504423,30673,3,32638,319
2,11,355089,567330,1,504423,30673,3,32638,319
3,9,363442,567330,1,504423,30673,3,32638,319
4,14,370513,567330,1,504423,30673,3,32638,319


In [16]:
df_user_features_test = pd.merge(df_user_click_log_test,df_ad_info_test,on='creative_id')

In [13]:
df_user_click_log_test.shape,df_user_features_test.shape

((33585512, 4), (33585512, 9))

In [27]:
df_user_features_test.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,3131989,645764,1,573314,58,2,14689,6
1,10,3142948,645764,1,573314,58,2,14689,6
2,14,3170643,645764,1,573314,58,2,14689,6
3,10,3194257,645764,1,573314,58,2,14689,6
4,21,3222925,645764,1,573314,58,2,14689,6


In [ ]:
df_user_features_test['creative_id'].drop_duplicates().shape,df_user_features_test['ad_id'].drop_duplicates().shape,df_user_features_test['advertiser_id'].drop_duplicates().shape

In [17]:
df_user_features = pd.concat((df_user_features_train,df_user_features_test))

In [18]:
del df_user_features_train
del df_user_features_test

In [19]:
df_user_features.shape

(63668283, 9)

## 提取分类特征

### 产品类别

In [20]:
df_user_features.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,15,320815,567330,1,504423,30673,3,32638,319
2,11,355089,567330,1,504423,30673,3,32638,319
3,9,363442,567330,1,504423,30673,3,32638,319
4,14,370513,567330,1,504423,30673,3,32638,319


In [40]:
df_user_features['click_times'].describe(percentiles=[0.05,0.25,0.5,0.75,0.95])

count    6.366828e+07
mean     1.064746e+00
std      3.300665e-01
min      1.000000e+00
5%       1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
95%      2.000000e+00
max      1.850000e+02
Name: click_times, dtype: float64

In [21]:
df_user_category_group = df_user_features.groupby(['user_id','product_category'])['click_times'].sum()
df_user_category_feature = df_user_category_group.unstack(fill_value=0)

In [22]:
df_user_category_feature.shape

(1900000, 18)

In [23]:
df_user_category_feature.head()

product_category,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
user_id,,,,,,,,,,,,,,,,,,
1,0,7,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,2
2,0,30,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,14
3,0,13,0,1,0,0,0,1,0,0,0,1,0,0,0,0,6,8
4,0,15,0,1,1,0,1,0,0,0,0,0,0,0,0,0,7,4
5,0,12,0,0,7,0,0,1,0,0,0,0,0,0,0,0,0,14


In [24]:
category_feature_names=[]
for x in df_user_category_feature.columns:
    category_feature_names.append("c_"+str(x))

In [25]:
df_user_category_feature.columns=category_feature_names

In [24]:
df_user_category_feature.head()

,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18
user_id,,,,,,,,,,,,,,,,,,
1,0,7,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,2
2,0,30,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,14
3,0,13,0,1,0,0,0,1,0,0,0,1,0,0,0,0,6,8
4,0,15,0,1,1,0,1,0,0,0,0,0,0,0,0,0,7,4
5,0,12,0,0,7,0,0,1,0,0,0,0,0,0,0,0,0,14


### 产业类别

In [26]:
df_user_industry_group = df_user_features.groupby(['user_id','industry'])['click_times'].sum()
df_user_industry_feature = df_user_industry_group.unstack(fill_value=0)

In [27]:
df_user_industry_feature.shape

(1900000, 332)

In [42]:
df_user_industry_feature.head()

,1,10,100,101,102,103,104,105,106,107,...,91,92,93,94,95,96,97,98,99,0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [34]:
'\\N'

'\\N'

In [38]:
# 将‘\\N’用0替代
i_columns = []
for x in df_user_industry_feature.columns:
    if x=='\\N':
        i_columns.append(0)
    else:
        i_columns.append(x)

In [41]:
df_user_industry_feature.columns = i_columns

In [43]:
industry_feature_names=[]
for x in df_user_industry_feature.columns:
    industry_feature_names.append("i_"+str(x))

In [44]:
df_user_industry_feature.columns=industry_feature_names

In [45]:
df_user_industry_feature.head()

,i_1,i_10,i_100,i_101,i_102,i_103,i_104,i_105,i_106,i_107,...,i_91,i_92,i_93,i_94,i_95,i_96,i_97,i_98,i_99,i_0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 合并分类特征

In [46]:
df_user_cate_feature = pd.merge(df_user_category_feature,df_user_industry_feature,on='user_id')

In [47]:
del df_user_category_group
del df_user_industry_group
del df_user_category_feature
del df_user_industry_feature

In [ ]:
gc.collect()

In [48]:
df_user_cate_feature.shape

(1900000, 350)

In [49]:
df_user_cate_feature.head()

,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,...,i_91,i_92,i_93,i_94,i_95,i_96,i_97,i_98,i_99,i_0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,7,0,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,30,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,13,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,8
4,0,15,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5,0,12,0,0,7,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


## 构造样本

In [34]:
df_sample_train = pd.merge(df_user_cate_feature,df_user_label,left_index=True,right_on='user_id')

In [35]:
df_sample_train.shape

(900000, 353)

In [36]:
df_sample_test = df_user_cate_feature.loc[df_user_click_log_test['user_id'].drop_duplicates().values]

In [37]:
del df_user_cate_feature

In [39]:
df_sample_test.shape

(1000000, 350)

In [40]:
df_sample_train.shape,df_sample_test.shape

((900000, 353), (1000000, 350))

In [77]:
df_sample_train.head()

,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,...,i_94,i_95,i_96,i_97,i_98,i_99,i_unknown,user_id,age,gender
0,0,7,0,0,5,0,0,0,0,0,...,0,0,0,0,0,0,1,1,4,1
1,0,30,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,2,10,1
2,0,13,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,8,3,7,2
3,0,15,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,9,4,5,1
4,0,12,0,0,7,0,0,1,0,0,...,0,0,0,0,0,0,1,5,4,1


In [78]:
df_sample_test.head()

,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,...,i_91,i_92,i_93,i_94,i_95,i_96,i_97,i_98,i_99,i_unknown
user_id,,,,,,,,,,,,,,,,,,,,,
3131989,0,14,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3131990,0,21,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3131994,0,14,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3131995,0,11,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3132000,0,7,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
df_sample_test = df_sample_test.rename_axis('user_id').reset_index()

In [42]:
df_sample_test.head()

,user_id,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,...,i_91,i_92,i_93,i_94,i_95,i_96,i_97,i_98,i_99,i_unknown
0,3131989,0,14,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3131990,0,21,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3131994,0,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3131995,0,11,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,4
4,3132000,0,7,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
feature_exclude_columns=['user_id','age','gender']

In [62]:
feature_columns = [x for x in df_sample_train.columns if x not in feature_exclude_columns]

In [63]:
len(feature_columns)

478

### 补充词向量特征（可选）

In [50]:
df_word2_vec = pd.read_csv("./data/user_word2_vec_128.csv")

In [51]:
df_word2_vec.shape

(1900000, 129)

In [46]:
df_word2_vec.head()

,user_id,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,...,w2v_118,w2v_119,w2v_120,w2v_121,w2v_122,w2v_123,w2v_124,w2v_125,w2v_126,w2v_127
0,1,-0.193146,-0.228589,-0.212177,0.502402,0.419108,-0.155171,-0.050705,-0.278514,-0.257985,...,-0.508188,-0.047964,0.173882,-0.233017,-0.106127,0.137993,0.364915,-0.258417,0.158040,-0.312584
1,2,-0.427113,-0.449429,0.155294,0.204986,0.057536,0.182058,-0.118976,-0.134916,0.312591,...,-0.383222,0.240402,0.421468,-0.091944,0.024992,0.344723,0.350942,-0.021163,-0.156320,0.067091
2,3,-0.290485,-0.123267,0.147264,0.294396,0.333517,-0.002210,-0.147300,-0.487144,0.015274,...,-0.172198,0.003900,0.099584,-0.028236,-0.022920,0.151936,0.563423,0.020704,-0.102448,0.014376
3,4,-0.258450,-0.598861,0.174674,0.272897,0.258750,0.102361,-0.110127,-0.355300,0.377514,...,-0.160130,0.139451,-0.130731,-0.239497,0.049233,0.100909,0.297746,0.005558,-0.144150,-0.023132
4,5,-0.403251,-0.223418,-0.140462,0.211255,0.477512,0.247194,-0.021708,-0.284819,-0.154669,...,-0.323597,-0.099868,0.383555,-0.227641,-0.094448,0.155718,0.490123,-0.108260,-0.032434,-0.036300


#### 融合特征

In [48]:
df_sample_train =  pd.merge(df_sample_train,df_word2_vec,on='user_id')

In [49]:
df_sample_train.shape

(900000, 481)

In [50]:
df_sample_test =  pd.merge(df_sample_test,df_word2_vec,on='user_id')

In [51]:
df_sample_test.shape

(1000000, 479)

## 清空冗余变量

In [197]:
del df_user_click_log_train
del df_user_click_log_test

In [53]:
gc.collect()

41198

## 建模

In [54]:
from sklearn.model_selection import train_test_split
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import accuracy_score

In [55]:
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

In [56]:
from sklearn.model_selection import cross_val_score

In [58]:
from sklearn.externals import joblib

### 性别

#### LGBM

In [73]:
lgbm_gender = LGBMClassifier(n_estimators=300,learning_rate=0.1,max_depth=8,num_leaves=127,subsample=0.8,colsample_bytree=0.8,min_split_gain=3,random_state=0,reg_alpha=0.05)

In [60]:
def model_test(clf,df_data,feature_columns,label_name,test_size=0.3,refit=False):
     '''
    建模测试
    Parameters
    ----------
    clf:sklearn model
    df_data：pandas.DataFrame,训练数据源
    feature_columns：list,特征列名
    label_name：str,标签列名
    test_size：验证集比例
    refit：boolean,测试完是否refit
    Returns
    ----------
    clf:sklearn model
    '''
    X_train, X_test, y_train, y_test = train_test_split(df_data[feature_columns],df_data[label_name],test_size=test_size,random_state=0)
    print("数据集划分完毕")
    clf.fit(X_train,y_train)
    print("模型训练完毕")
    y_test_predict = clf.predict(X_test)
    y_train_predict = clf.predict(X_train)
    print("训练集：%f,测试集：%f" % (accuracy_score(y_train,y_train_predict),accuracy_score(y_test,y_test_predict)))
    if refit is True:
        clf.fit(df_data[feature_columns],df_data[label_name])
    return clf

产品+产业类别+word2vec

In [64]:
lgbm_gender = model_test(lgbm_gender,df_sample_train,feature_columns=feature_columns,label_name='gender')

数据集划分完毕
模型训练完毕
训练集：0.946217,测试集：0.922156


refit

In [74]:
lgbm_gender.fit(df_sample_train[feature_columns],df_sample_train['gender'])

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
               importance_type='split', learning_rate=0.1, max_depth=8,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=3,
               n_estimators=300, n_jobs=-1, num_leaves=127, objective=None,
               random_state=0, reg_alpha=0.05, reg_lambda=0.0, silent=True,
               subsample=0.8, subsample_for_bin=200000, subsample_freq=0)

test

In [91]:
X_train, X_test, y_train, y_test = train_test_split(df_sample_train[feature_columns],df_sample_train['gender'],test_size=0.3,random_state=0)

In [92]:
X_train.shape,X_test.shape

((630000, 350), (270000, 350))

In [93]:
y_train.value_counts()

1    421968
2    208032
Name: gender, dtype: int64

In [94]:
y_test.value_counts()

1    180642
2     89358
Name: gender, dtype: int64

只用产品类别:0.68

产品+产业类别

In [117]:
accuracy_score(y_train,y_train_predict)

0.8518460317460318

In [118]:
accuracy_score(y_test,y_test_predict)

0.8417074074074075

5折交叉验证

In [101]:
%time cv_score = cross_val_score(lgbm_gender,X_train,y_train,scoring='accuracy',cv=5)

CPU times: user 23min 19s, sys: 1min 4s, total: 24min 24s
Wall time: 4min 2s


In [103]:
cv_score.mean()

0.8414222222222222

#### 贝叶斯

In [120]:
clf_bnb = BernoulliNB()

In [123]:
clf_bnb.fit(X_train,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [129]:
y_test_predict = clf_bnb.predict(X_test)

In [130]:
y_train_predict = clf_bnb.predict(X_train)

产品+产业类别

In [131]:
accuracy_score(y_train,y_train_predict)

0.7970063492063492

In [132]:
accuracy_score(y_test,y_predict)

0.7971185185185186

In [159]:
gc.collect()

70391

### 年龄

#### LGBM

In [75]:
lgbm_age = LGBMClassifier(n_estimators=300,learning_rate=0.1,max_depth=8,num_leaves=127,subsample=0.8,colsample_bytree=0.8,min_split_gain=3,random_state=0,reg_alpha=0.02)

产品+产业类别+word2vec

In [67]:
lgbm_age = model_test(lgbm_age,df_sample_train,feature_columns=feature_columns,label_name='age')

数据集划分完毕
模型训练完毕
训练集：0.552656,测试集：0.384619


refit

In [76]:
lgbm_age.fit(df_sample_train[feature_columns],df_sample_train['age'])

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
               importance_type='split', learning_rate=0.1, max_depth=8,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=3,
               n_estimators=300, n_jobs=-1, num_leaves=127, objective=None,
               random_state=0, reg_alpha=0.02, reg_lambda=0.0, silent=True,
               subsample=0.8, subsample_for_bin=200000, subsample_freq=0)

test

In [147]:
X_train, X_test, y_train, y_test = train_test_split(df_sample_train[feature_columns],df_sample_train['age'],test_size=0.3,random_state=0)

In [161]:
y_test_predict = lgbm_age.predict(X_test)
y_train_predict = lgbm_age.predict(X_train)

只用产品类别

In [112]:
accuracy_score(y_test,y_predict)

0.23807777777777778

产品+产业类别

In [151]:
accuracy_score(y_train,y_train_predict)

0.343968253968254

In [152]:
accuracy_score(y_test,y_test_predict)

0.3012

产品+产业类别:balanced(多分类class_weight:balanced)

In [157]:
accuracy_score(y_train,y_train_predict)

0.2971761904761905

In [158]:
accuracy_score(y_test,y_test_predict)

0.2457148148148148

#### 贝叶斯

In [137]:
clf_bnb_age = BernoulliNB()

In [138]:
clf_bnb_age.fit(X_train,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [141]:
y_test_predict = clf_bnb_age.predict(X_test)

In [142]:
y_train_predict = clf_bnb_age.predict(X_train)

产品+产业类别

In [144]:
accuracy_score(y_train_predict,y_train)

0.2665571428571429

In [147]:
accuracy_score(y_test,y_test_predict)

0.2665148148148148

## 模型固化

In [77]:
joblib.dump(lgbm_gender,'./model/lgbm_gender_w2v128_cate350.pkl')

['./model/lgbm_gender_w2v128_cate350.pkl']

In [179]:
lgbm_gender = joblib.load("./model/lgbm_gender_w2v128_cate350.pkl")

In [180]:
y_test_predict = lgbm_gender.predict(X_test)
y_train_predict = lgbm_gender.predict(X_train)

In [ ]:
accuracy_score(y_train,y_train_predict),accuracy_score(y_test,y_test_predict)

In [ ]:
joblib.dump(lgbm_age,'./model/lgbm_age_w2v128_cate350.pkl')

## 测试集输出结果

性别

In [78]:
train_result_gender = lgbm_gender.predict(df_sample_train[feature_columns])

In [79]:
accuracy_score(df_sample_train['gender'],train_result_gender)

0.9407177777777778

In [ ]:
df_sample_train['predicted_gender']=train_result_gender

In [80]:
test_result_gender = lgbm_gender.predict(df_sample_test[feature_columns])

In [124]:
test_result_gender

array([1, 2, 1, ..., 1, 1, 1])

In [81]:
df_sample_test['predicted_gender'] = test_result_gender

年龄

In [85]:
train_result_age = lgbm_age.predict(df_sample_train[feature_columns])

In [86]:
accuracy_score(df_sample_train['age'],train_result_age)

0.5245644444444445

In [ ]:
df_sample_train['predicted_age']=train_result_age

In [87]:
test_result_age = lgbm_age.predict(df_sample_test[feature_columns])

In [167]:
test_result_age

array([6, 2, 4, ..., 3, 3, 3])

In [92]:
df_sample_test['predicted_age'] = test_result_age

导出结果

In [93]:
df_sample_test[['user_id','predicted_age','predicted_gender']].to_csv("./result/submission.csv",index=False)

In [171]:
df_sample_test_result = pd.read_csv("./result/submission.csv")

In [172]:
df_sample_test_result.shape

(1000000, 3)

观察测试集和训练集预测标签分布

In [130]:
df_sample_train['gender'].value_counts()

1    602610
2    297390
Name: gender, dtype: int64

In [ ]:
df_sample_train['predicted_gender'].value_counts()

In [82]:
df_sample_test['predicted_gender'].value_counts()

1    679278
2    320722
Name: predicted_gender, dtype: int64

In [ ]:
df_sample_train['age'].value_counts()

In [ ]:
df_sample_train['predicted_age'].value_counts()

In [ ]:
df_sample_test['predicted_age'].value_counts()